In [1]:
import pandas as pd
import numpy as np
import json
import csv
from datetime import datetime

In [2]:
usersOld = pd.read_csv('data-live/users_prod.csv')
restsOld = pd.read_csv('data-live/restaurants_prod.csv')
mealsOld = pd.read_csv('data-live/meals_prod.csv')
hospsOld = pd.read_csv('data-live/hospitals_prod.csv')
# infosOld = pd.read_csv('data-live/mealinfo_prod.csv')
seedData = pd.read_csv('data-live/seed_data.csv')

In [3]:
userDict = {}
restDict = {}
mealDict = {}
infoDict = {}
hospDict = {}
def userToDict(r):
    userDict[r.username] = {"id":r.id}
def restToDict(r):
    restDict[r.restName] = {"id":r.id, "userId":r.userId}
def mealToDict(r):
    mealDict[r.mealName] = {"id":r.id, "restName":r.restName, "userId":r.userId, "mealName" : r.mealName}
def infoToDict(r):
    infoDict[r.type] = {"id":r.id}
def hospToDict(r):
    hospDict[r.hospName] = {"id":r.id}

v = usersOld.apply(userToDict, axis=1)
v = restsOld.apply(restToDict, axis=1)
v = mealsOld.apply(mealToDict, axis=1)
# v = infosOld.apply(infoToDict, axis=1)
v = hospsOld.apply(hospToDict, axis=1)

In [4]:
# Convert the date correctly
def getDate(r):
    time = ''
    if(r['meal'] == 'Lunch'):
        time = '12:00:00-04'
    elif(r['hospital'] == 'Hospital of the University of Pennsylvania (HUP)'):
        time = '20:30:00-04'
    elif(r['meal'] == 'Dinner'):
        time = '20:00:00-04'
    else:
        print(r)
    date = ["%02d" % int(v) for v in r['date'].split('/')]
    date = "%s-%s-%s" % (date[2], date[0], date[1])
    return ("%s %s" % (date, time))

seedData['timeslot'] = seedData[['hospital', 'date', 'meal']].apply(getDate, axis=1)
seedData = seedData.fillna(value=False)

In [5]:
np.unique(np.concatenate([seedData['mealDiet%i' %i].unique() for i in range(1, 5)]))

array([False, 'Dairy-Free', 'Gluten-Free', 'Gluten-Free, Dairy-Free',
       'Vegan', 'Vegan, Gluten-Free, Dairy-Free',
       'Vegan, Vegetarian, Dairy-Free', 'Vegan, Vegetarian, Gluten-Free',
       'Vegan, Vegetarian, Gluten-Free, Dairy-Free', 'Vegetarian',
       'Vegetarian, Gluten-Free', 'Vegetarian, Gluten-Free, Dairy-Free'],
      dtype=object)

In [6]:
dietLookup = {
  "V" : "Vegan",
  "VG" : "Vegetarian",
  "GF" : "Gluten Free",
  "DF" : "Dairy Free",
  "Vegan": "Vegan",
  "Vegetarian" : "Vegetarian",
  "Dairy-Free": "Dairy Free",
  "Gluten-Free" : "Gluten Free"
}
allergenLookup = {
  'Milk/Dairy' : 'milk/dairy',
  'Milk/dairy' : "milk/dairy",
  'Eggs' : "eggs",
  'Eggs in aioli' : 'eggs',
  'Pumpkin seeds (seed allergy)' : "pumpkin seeds",
  'Sesame' : 'sesame',
  'Sesame Oil' : 'sesame',
  'Soy' : 'soy',
  'Dairy' : 'milk/dairy',
  'Tree nuts' : 'tree nuts',
  'Fish': 'fish',
  'Shellfish' : 'shellfish',
  'Wheat' : 'wheat',
  'Peanuts': 'peanuts'
}

In [7]:
# Build the restaurant struct
restStruct = {}

def dateCompare(t0, t1='2020-05-03 00:00:00-04'):
    v1 = datetime.strptime(t0, '%Y-%m-%d %H:%M:%S-04')
    v2 = datetime.strptime(t1, '%Y-%m-%d %H:%M:%S-04')
    return v1 > v2

def buildStruct(r):
    restName = r['restName']
    restStruct[restName] = restStruct.get(restName, {"mealIds": {}, "meals": [], "timeslots": [], "menus": []})
    # Create the timeslot and add to list
    timeslotId = str(("%s-%s-%s" % (restName, r['timeslot'], r['hospital'])).__hash__())
    timeslot = {
        "id": timeslotId,
        "hospitalId": hospDict[r['hospital']]['id'],
        "restaurantId": restDict[r['restName']]['id'],
        "date": r['timeslot'],
        "userId": restDict[r['restName']]['userId']
    }
    if dateCompare(r['timeslot']):
        restStruct[restName]['timeslots'].append(timeslot) 
    # Generate the meals/menus
    for i in range(1, 5):
        mealName = r["mealName%d" % i]
        if(mealName):
            allergens, dietaryRestrictions = [], []
            if(r['mealAllergens%d' %i]):
                allergens = [v.lstrip().rstrip() for v in r['mealAllergens%d' %i].split(',')]
                allergens = [allergenLookup[a] for a in allergens]
            if(r['mealDiet%d' % i]):
                dietaryRestrictions = [v.lstrip().rstrip() for v in r['mealDiet%d' % i].split(',')]
                dietaryRestrictions = [dietLookup[d] for d in dietaryRestrictions]
        # Check if meal exists else add
            if(not mealName in mealDict):
                meal = {
                    "name": mealName,
                    "description": r['mealDesc%d' %i],
                    "allergens" : allergens,
                    "dietaryRestrictions": dietaryRestrictions,
                    "mealinfoId": r['meal'].lower() + 'New',
                    "restaurantId": restDict[r['restName']]['id'],
                    "userId": restDict[r['restName']]['userId']
                }
                mealDict[mealName] = meal
                restStruct[restName]['meals'].append(meal)
                print("Appending: " + mealName)
                print(restStruct[restName]['meals'])
            # Add the menu to the list
            menu = {
                "timeslotId": timeslotId,
                "mealinfoId": r['meal'].lower() + 'New',
                "mealName": mealName,
                "mealDescription": r['mealDesc%d' %i],
                "allergens" : allergens,
                "dietaryRestrictions": dietaryRestrictions,
                "userId": restDict[r['restName']]['userId']
            }
            if dateCompare(r['timeslot']):
                restStruct[restName]['menus'].append(menu)
            
s = seedData.apply(buildStruct, axis=1)

Appending: Dorada Salad With Braised Pork
[{'allergens': [], 'restaurantId': '571ab7c1-e8cd-5d8f-af51-7cbc35eed917', 'name': 'Dorada Salad With Braised Pork', 'userId': '949356c0-c4e8-5133-ba84-45b3fd262d19', 'mealinfoId': 'lunchNew', 'dietaryRestrictions': ['Gluten Free', 'Dairy Free'], 'description': 'Braised buttery pork butt served over a bed of local greens with roasted carrot, heart of palm, green beans & toasted sunflower seeds . Served with mustard vinaigrette. '}]
Appending: Dorada Salad
[{'allergens': [], 'restaurantId': '571ab7c1-e8cd-5d8f-af51-7cbc35eed917', 'name': 'Dorada Salad With Braised Pork', 'userId': '949356c0-c4e8-5133-ba84-45b3fd262d19', 'mealinfoId': 'lunchNew', 'dietaryRestrictions': ['Gluten Free', 'Dairy Free'], 'description': 'Braised buttery pork butt served over a bed of local greens with roasted carrot, heart of palm, green beans & toasted sunflower seeds . Served with mustard vinaigrette. '}, {'allergens': ['soy'], 'restaurantId': '571ab7c1-e8cd-5d8f-af5

In [8]:
restStruct

{'Baology': {'mealIds': {},
  'meals': [{'allergens': [],
    'description': 'Taiwanese-style Bento served with Roasted Pastured Chicken with local veggies over Forbidden Rice',
    'dietaryRestrictions': ['Gluten Free', 'Dairy Free'],
    'mealinfoId': 'lunchNew',
    'name': 'Roasted Pastured Chicken Bian Dang/Bento',
    'restaurantId': 'c22d4fbb-c9df-5cea-a11b-79ecf3c05e43',
    'userId': 'ba4a1061-1740-55ca-86f5-1c5ea777fa5d'},
   {'allergens': [],
    'description': 'Taiwanese-style Bento served with Roasted Heritage Pork with local veggies over Forbidden Rice',
    'dietaryRestrictions': ['Gluten Free', 'Dairy Free'],
    'mealinfoId': 'lunchNew',
    'name': 'Heritage Pork Bian Dang/Bento',
    'restaurantId': 'c22d4fbb-c9df-5cea-a11b-79ecf3c05e43',
    'userId': 'ba4a1061-1740-55ca-86f5-1c5ea777fa5d'},
   {'allergens': [],
    'description': 'Taiwanese-style Bento served with Roasted Local Mushrooms with local veggies over Forbidden Rice',
    'dietaryRestrictions': ['Vegan',


In [9]:
dataOut = json.dumps(restStruct, indent=2)
with open("update_data.json", "w") as outfile: 
    outfile.write(dataOut) 

In [10]:
# Output the csvs
with open("timeslot_data.csv", "w") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['id', 'restaurantId', 'hospitalId', 'userId', 'date'])
    for restName in restStruct:
        for ts in restStruct[restName]['timeslots']:
            writer.writerow([ts['id'], ts['restaurantId'], ts['hospitalId'], ts['userId'], ts['date']])

In [11]:
# Output the csvs
with open("menu_data.csv", "w") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['timeslotId', 'mealinfoId', 'userId', 'mealName', 'mealDescription', 'allergens', 'dietaryRestrictions'])
    for restName in restStruct:
        for ts in restStruct[restName]['menus']:
            writer.writerow([ts['timeslotId'], ts['mealinfoId'], ts['userId'], ts['mealName'], ts['mealDescription'], ts['allergens'], ts['dietaryRestrictions']])
            

In [13]:
# Output the csvs
with open("meal_data.csv", "w") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['restaurantId', 'mealinfoId', 'userId', 'mealName', 'mealDescription', 'allergens', 'dietaryRestrictions'])
    for restName in restStruct:
        for ts in restStruct[restName]['meals']:
            writer.writerow([ts['restaurantId'], ts['mealinfoId'], ts['userId'], ts['name'], ts['description'], ts['allergens'], ts['dietaryRestrictions']])
            